# Imports

In [60]:
import pandas as pd
import numpy as np

from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_log_error

# Data preparation

In [2]:
train = pd.read_csv('./data/train.csv', parse_dates=['date'])
test = pd.read_csv('./data/test.csv', parse_dates=['date'])
holidays_events = pd.read_csv('./data/holidays_events.csv', parse_dates=['date'])
oil = pd.read_csv('./data/oil.csv', parse_dates=['date'])
stores = pd.read_csv('./data/stores.csv')
transactions = pd.read_csv('./data/transactions.csv', parse_dates=['date'])
data = pd.read_csv('./data/data.csv')

### Check if there are missing dates in the train data

In [3]:
# Get the date range from earliest to latest in train data
date_range = pd.date_range(start=train['date'].min(), end=train['date'].max(), freq='D')

# Get unique dates in train data
train_dates = train['date'].unique()

# Find missing dates
missing_dates = date_range.difference(pd.DatetimeIndex(train_dates))

print(f"Total dates in range: {len(date_range)}")
print(f"Unique dates in train: {len(train_dates)}")
print(f"Missing dates: {len(missing_dates)}")
print(f"\nMissing dates:\n{missing_dates}")


Total dates in range: 1688
Unique dates in train: 1684
Missing dates: 4

Missing dates:
DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)


In [4]:
# Create a complete date range from train data
date_range = pd.date_range(start=train['date'].min(), end=train['date'].max(), freq='D')

# Get all unique combinations of store number and product family
stores_list = train['store_nbr'].unique()
unique_product_families = train['family'].unique()

# Create a MultiIndex with all combinations of date, store_nbr, and family
complete_index = pd.MultiIndex.from_tuples(
    product(date_range, stores_list, unique_product_families),
    names=['date', 'store_nbr', 'family']
)

# Create a complete dataframe
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Merge with original train data
train_complete = complete_df.merge(
    train,
    on=['date', 'store_nbr', 'family'],
    how='left'
)

# Fill missing values i.e. sales and onpromotion with 0
train_complete['sales'] = train_complete['sales'].fillna(0)
train_complete['onpromotion'] = train_complete['onpromotion'].fillna(0)

# 4 missing dates, 54 stores, 33 product families i.e. 4 * 54 * 33 = 7128
print(f"Added records: {len(train_complete) - len(train)}")

train = train_complete

Added records: 7128


### Check missing oil dates

In [5]:
# Get the date range from earliest to latest in train and test data
date_range = pd.date_range(start=train['date'].min(), end=test['date'].max(), freq='D')

# Get unique dates in oil data
oil_dates = oil['date'].unique()

# Find missing dates
missing_dates = date_range.difference(pd.DatetimeIndex(oil_dates))

print(f"Total dates in range: {len(date_range)}")
print(f"Unique dates in oil: {len(oil_dates)}")
print(f"Missing dates: {len(missing_dates)}")

Total dates in range: 1704
Unique dates in oil: 1218
Missing dates: 486


In [6]:
# Create complete date range for the data from train and test data
date_range = pd.date_range(start=train['date'].min(), end=test['date'].max(), freq='D')

# Create complete dataframe with all dates
oil_complete = pd.DataFrame({'date': date_range})

# Merge with original oil data
oil_complete = oil_complete.merge(
    oil, 
    on='date', 
    how='left'
)

# Forward fill (use last known price for missing dates)
oil_complete['dcoilwtico'] = oil_complete['dcoilwtico'].ffill()

# Backward fill for any remaining NaNs at the start
oil_complete['dcoilwtico'] = oil_complete['dcoilwtico'].bfill()

# Update oil dataframe
print(f"Number of records after filling missing oil data: {len(oil_complete)}")
print(f"Added records: {len(oil_complete) - len(oil)}")

oil = oil_complete

Number of records after filling missing oil data: 1704
Added records: 486


### Check transactions

In [7]:
# Total transactions count
total_transactions_count = len(train.groupby(["date", "store_nbr"])['sales'].sum())

print(f"Missing records from transactions: {total_transactions_count - len(transactions)}")


store_sales = train.groupby(['date', 'store_nbr'])['sales'].sum().reset_index()

# Merge transactions with sales
transactions_complete = transactions.merge(
    store_sales,
    on=['date', 'store_nbr'],
    how='outer'
).sort_values(by=['date', 'store_nbr'], ignore_index=True)

# For dates where sales are 0, set transactions to 0
transactions_complete.loc[transactions_complete["sales"].eq(0), "transactions"] = 0

# Interpolate missing transactions for each store
transactions_complete['transactions'] = transactions_complete.groupby('store_nbr')['transactions'].transform(
    lambda x: x.interpolate(method='linear')
)

# Round transactions to remove fractions from interpolation
transactions_complete['transactions'] = transactions_complete['transactions'].round().astype(int)

# Drop sales column
transactions_complete = transactions_complete.drop('sales', axis=1)

print(f"Added transactions: {len(transactions_complete) - len(transactions)}")

transactions = transactions_complete

Missing records from transactions: 7664
Added transactions: 7664


### Holidays

In [54]:
holidays_events["description"] = holidays_events.apply(
    lambda x: x["description"].strip().lower().replace(x["locale_name"].lower(), ""), axis=1 # Remove names
).apply(
    lambda x: "futbol" if "futbol" in x else x # Only keep futbol
).replace(r'\b(de|del|puente|traslado|recupero)\b', '', regex=True
).replace(r'[+-]\d+', '', regex=True # Remove digits with leading + or -
).str.strip()

holidays_events = holidays_events[holidays_events["type"] != "Transfer"] # Transfer holidays are basically not actual holidays

In [ ]:
def check_holiday(row, holiday_events):
    holiday_rows = holiday_events[holiday_events['date'] == row['date']]
    
    # There are no holidays
    if holiday_rows.empty:
        return 0
    
    # Check for National holiday
    if not holiday_rows[holiday_rows['locale'] == 'National'].empty:
        return 1
    
    # Check for Regional holiday (state must match)
    regional_holidays = holiday_rows[holiday_rows['locale'] == 'Regional']
    if not regional_holidays.empty:
        if not regional_holidays[regional_holidays['locale_name'] == row['state']].empty:
            return 1
    
    # Check for Local holiday (city must match)
    local_holidays = holiday_rows[holiday_rows['locale'] == 'Local']
    if not local_holidays.empty:
        if not local_holidays[local_holidays['locale_name'] == row['city']].empty:
            return 1

    return 0

In [ ]:
data = pd.concat(
    [train, test], axis=0, ignore_index=True
).merge(
    transactions, on=['date', 'store_nbr'], how='left'  
).merge(
    oil, on='date', how='left'
).merge(
    stores, on='store_nbr', how='left'
)

# This can be optimized by using a more efficient approach (e.g. merging)
data["holiday"] = data.apply(
    lambda x: check_holiday(x, holidays_events), axis=1
)

### Creating one and two day lags for dcoilwtico

In [69]:
# Sort by store_nbr, family, and date to ensure correct order within each group
data = data.sort_values(['store_nbr', 'family', 'date'])

# Create lag-1 column within each store-family combination
data['lag-1-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['dcoilwtico'].shift(1)

# Create lag-2 column (2 days ago)
data['lag-2-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['dcoilwtico'].shift(2)

# Backfill NaNs at the start of each group
data['lag-1-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['lag-1-dcoilwtico'].bfill()
data['lag-2-dcoilwtico'] = data.groupby(['store_nbr', 'family'])['lag-2-dcoilwtico'].bfill()
data = data.sort_values('date')

data.to_csv('./data/data.csv', index=False)

### Extract date features

In [49]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_year'] = data['date'].dt.dayofyear
data['week_of_year'] = data['date'].dt.isocalendar().week
data['is_weekend'] = (data['date'].dt.dayofweek >= 5).astype(int)

### Label Encoding for categorical features

In [50]:
# Get all unique values for each categorical column from data
all_product_families = data['family'].unique()
all_cities = data['city'].unique()
all_states = data['state'].unique()
all_types = data['type'].unique()

# Initialize label encoders
le_product_family = LabelEncoder()
le_city = LabelEncoder()
le_state = LabelEncoder()
le_type = LabelEncoder()

# Fit on all unique values
le_product_family.fit(all_product_families)
le_city.fit(all_cities)
le_state.fit(all_states)
le_type.fit(all_types)

# Transform data dataframe
data['family'] = le_product_family.transform(data['family'])
data['city'] = le_city.transform(data['city'])
data['state'] = le_state.transform(data['state'])
data['type'] = le_type.transform(data['type'])

print("Label encoding complete!")
print(f"Family: {len(all_product_families)} categories")
print(f"City: {len(all_cities)} categories")
print(f"State: {len(all_states)} categories")
print(f"Type: {len(all_types)} categories")

Label encoding complete!
Family: 33 categories
City: 22 categories
State: 16 categories
Type: 5 categories


### Splitting test and train data

In [51]:
# Split based on date
train = data[data['date'] < '2017-08-16'].copy()
test = data[data['date'] >= '2017-08-16'].copy()

# Save test IDs before dropping
test_ids = test['id'].copy()

# Drop transactions column from both
train = train.drop('transactions', axis=1)
test = test.drop('transactions', axis=1)

# Drop date column since we have date features
train = train.drop('date', axis=1)
test = test.drop('date', axis=1)

# Drop id column
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

# Drop sales column only from test
test = test.drop('sales', axis=1)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Test IDs saved: {len(test_ids)}")

Train shape: (3008016, 19)
Test shape: (28512, 18)
Test IDs saved: 28512


# Model training

In [ ]:
# Separate features and target
X_train = train.drop('sales', axis=1)
y_train = train['sales']

X_test = test 

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Target: {y_train.shape}")

Training set: (3008016, 18)
Test set: (28512, 18)
Target: (3008016,)


In [ ]:
# Initialize Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# Train the model
print("Training Random Forest model...")
rf_model.fit(X_train, y_train)

# Make predictions on training set to evaluate
y_train_pred = rf_model.predict(X_train)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmsle_score = np.sqrt(mean_squared_log_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print(f"\nTraining Metrics:")
print(f"RMSLE: {rmsle_score:.2f}")
print(f"RMSE: {train_rmse:.2f}")
print(f"MAE: {train_mae:.2f}")
print(f"R² Score: {train_r2:.4f}")

Training Random Forest model...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 11.2min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 11.2min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    7.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    7.1s



Training Metrics:
RMSE: 195.05
MAE: 40.76
R² Score: 0.9686


[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   19.2s finished


In [ ]:
# Make predictions on test set
y_test_pred = rf_model.predict(X_test)

print(f"Test predictions generated: {len(y_test_pred)}")

Test predictions generated: 28512
Sample predictions: [7.64449679e-01 1.46636656e+02 3.17176471e-01 1.15108877e+00
 2.18462200e+02 3.84661752e+03 9.27108390e+03 1.37646594e+02
 3.09251844e+03 4.57692364e+02]


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


# Create submission

In [ ]:
def create_submission(predictions, test_ids, filename='submission.csv'):
    submission = pd.DataFrame({
        'id': test_ids.values.astype(int),
        'sales': predictions
    })
    
    submission.to_csv(filename, index=False)
    return submission

In [59]:
# Create submission file using the saved test IDs
submission = create_submission(y_test_pred, test_ids, 'submission.csv')